In [1]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import os

In [2]:
ratings = pd.read_csv('C:/Users/Lenovo IdeaPad JQ/Dev/movie-recommender/movie-data/ratings.csv')
credits = pd.read_csv('C:/Users/Lenovo IdeaPad JQ/Dev/movie-recommender/movie-data/credits.csv')
metadata = pd.read_csv('C:/Users/Lenovo IdeaPad JQ/Dev/movie-recommender/movie-data/movies_metadata.csv')
links = pd.read_csv('C:/Users/Lenovo IdeaPad JQ/Dev/movie-recommender/movie-data/links.csv')
credits = pd.read_csv('C:/Users/Lenovo IdeaPad JQ/Dev/movie-recommender/movie-data/credits.csv')
keywords = pd.read_csv('C:/Users/Lenovo IdeaPad JQ/Dev/movie-recommender/movie-data/keywords.csv')

C:\Users\Lenovo IdeaPad JQ\AnacondaFolder\envs\justin-env\lib\site-packages\IPython\core\interactiveshell.py:3051: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
metadata_df = metadata.copy()
metadata_df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [5]:
metadata_df.shape

(45466, 24)

In [6]:
metadata_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
adult                    45466 non-null object
belongs_to_collection    4494 non-null object
budget                   45466 non-null object
genres                   45466 non-null object
homepage                 7782 non-null object
id                       45466 non-null object
imdb_id                  45449 non-null object
original_language        45455 non-null object
original_title           45466 non-null object
overview                 44512 non-null object
popularity               45461 non-null object
poster_path              45080 non-null object
production_companies     45463 non-null object
production_countries     45463 non-null object
release_date             45379 non-null object
revenue                  45460 non-null float64
runtime                  45203 non-null float64
spoken_languages         45460 non-null object
status                   45379 non-null objec

In [7]:
columns = list(metadata_df.columns)
columns

['adult',
 'belongs_to_collection',
 'budget',
 'genres',
 'homepage',
 'id',
 'imdb_id',
 'original_language',
 'original_title',
 'overview',
 'popularity',
 'poster_path',
 'production_companies',
 'production_countries',
 'release_date',
 'revenue',
 'runtime',
 'spoken_languages',
 'status',
 'tagline',
 'title',
 'video',
 'vote_average',
 'vote_count']

In [8]:
# IMDB formula to get weighted movie score WR == (v/(v+m) R) + (m/(m+v) C)

# v -> votes
v = metadata_df['vote_count']
# m -> minimum votes required
m = metadata_df['vote_count'].quantile(0.90)
# R -> average movie rating
R = metadata_df['vote_average']
# C -> mean vote
C = metadata_df['vote_average'].mean()

In [9]:
def get_weighting_rate(df,C=C,m=m):
    
    R = df['vote_average']
    v= df['vote_count']
    wr =  (v/(v+m) * R) + (m/(m+v) * C)
    return wr

#filtering the movies to be scored 
m_movies = metadata_df.loc[metadata_df['vote_count'] >= m]
# Compute the score for every movie in the dataset 

m_movies['score']=m_movies.apply(get_weighting_rate,axis=1)

#show the movie title and its score  
m_movies[['title','score']].head(10)

C:\Users\Lenovo IdeaPad JQ\AnacondaFolder\envs\justin-env\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


,title,score
0,Toy Story,7.640253
1,Jumanji,6.820293
4,Father of the Bride Part II,5.660700
5,Heat,7.537201
8,Sudden Death,5.556626
9,GoldenEye,6.483983
10,The American President,6.107000
11,Dracula: Dead and Loving It,5.664630
12,Balto,6.693333
15,Casino,7.567740


In [10]:
# Get the top 20 movies
m_movies = m_movies.sort_values('score', ascending=False)
m_movies[['title','vote_count','vote_average','score']].head(20)

,title,vote_count,vote_average,score
314,The Shawshank Redemption,8358.0,8.5,8.445869
834,The Godfather,6024.0,8.5,8.425439
10309,Dilwale Dulhania Le Jayenge,661.0,9.1,8.421453
12481,The Dark Knight,12269.0,8.3,8.265477
2843,Fight Club,9678.0,8.3,8.256385
292,Pulp Fiction,8670.0,8.3,8.251406
522,Schindler's List,4436.0,8.3,8.206639
23673,Whiplash,4376.0,8.3,8.205404
5481,Spirited Away,3968.0,8.3,8.196055
2211,Life Is Beautiful,3643.0,8.3,8.187171


In [32]:
overviews = metadata_df['overview']
overviews

0        Led by Woody, Andy's toys live happily in his ...
1        When siblings Judy and Peter discover an encha...
2        A family wedding reignites the ancient feud be...
3        Cheated on, mistreated and stepped on, the wom...
4        Just when George Banks has recovered from his ...
                               ...                        
45461          Rising and falling between a man and woman.
45462    An artist struggles to finish his work while a...
45463    When one of her hits goes wrong, a professiona...
45464    In a small town live two brothers, one a minis...
45465    50 years after decriminalisation of homosexual...
Name: overview, Length: 45466, dtype: object

In [33]:
overviews.shape

(45466,)

In [34]:
overviews = overviews[:15000]
overviews.shape

(15000,)

In [35]:
# TF-IDF vectorizer (https://machinelearningmastery.com/prepare-text-data-machine-learning-scikit-learn/)
# https://en.wikipedia.org/wiki/Tf%E2%80%93idf
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english')
overviews = overviews.fillna('')
tfidf_matrix = tfidf.fit_transform(overviews)
tfidf_matrix.shape

(15000, 40226)

In [36]:
# Calculate similarity scores (cosine similarity, or dot product)
from sklearn.metrics.pairwise import linear_kernel

cos_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cos_sim.shape

(15000, 15000)

In [37]:
# indices = pd.Series(metadata_df[:45466].index, index=metadata_df['title'][:45466]).drop_duplicates()
indices = pd.Series(metadata_df[:23000].index, index=metadata_df['title'][:23000]).drop_duplicates()
indices[:10]

title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
Heat                           5
Sabrina                        6
Tom and Huck                   7
Sudden Death                   8
GoldenEye                      9
dtype: int64

In [38]:
def get_recommendations(title, cos_sim=cos_sim):
    index = indices[title]
    
    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cos_sim[index]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:21]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return metadata_df['title'].iloc[movie_indices]

In [39]:
get_recommendations('Star Wars')

1154                 The Empire Strikes Back
1167                      Return of the Jedi
1267                            Mad Dog Time
5187                     The Triumph of Love
309                        The Swan Princess
461                     Hot Shots! Part Deux
3502                           Shanghai Noon
1987                         Sleeping Beauty
13735               The Flame of New Orleans
11322                        Dante's Inferno
10849                 Leprechaun 4: In Space
3036                The 7th Voyage of Sinbad
7149                     The Thief of Bagdad
13311    Fire and Ice: The Dragon Chronicles
7335                         The Magic Flute
3046                             Topsy-Turvy
573                         Princess Caraboo
9958                       Who's Your Daddy?
5784                  Attila flagello di Dio
2188            History of the World: Part I
Name: title, dtype: object

In [20]:
mo = metadata_df['vote_count'].quantile(0.50)

def get_recommendations_v2(title, cos_sim=cos_sim):
    index = indices[title]
    
    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cos_sim[index]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:21]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return metadata_df['title'].iloc[movie_indices].loc[metadata_df['vote_count'] >= mo]

In [24]:
reco = get_recommendations_v2('Star Trek')
reco

1327           Star Trek IV: The Voyage Home
1324         Star Trek V: The Final Frontier
1326     Star Trek III: The Search for Spock
40460                  For the Love of Spock
6575                              Paper Moon
9059             Pumpkinhead II: Blood Wings
34176                      American Warships
32022                         Charlie's Farm
32610                         Humanity's End
5954                The Last American Virgin
38363                        Ratchet & Clank
20922                Star Trek Into Darkness
8555                           The Music Man
23358                                   Lucy
20540                                Phantom
38571                            Restoration
Name: title, dtype: object

In [21]:
import pickle
import os.path

# pickle_filename = 'content_model.pkl'

# # with open(pickle_filename, 'wb') as f:
# #     pickle.dump(cos_sim, f)

# with open(pickle_filename, 'wb') as pfile:
#     pickle.dump(cos_sim, pfile, protocol=pickle.HIGHEST_PROTOCOL)

In [88]:
with open('content_model.pkl', 'rb') as infile:
    x = pickle.load(infile)
x.shape

(25000, 25000)

In [40]:
import _pickle as cPickle

pickle_filename = 'cos_sim.pkl'

with open(pickle_filename, 'wb') as f:
    pickle.dump(cos_sim, f, protocol=pickle.HIGHEST_PROTOCOL)

In [31]:
with open(pickle_filename, 'rb') as f:
    m = pickle.load(f, protocol=pickle.HIGHEST_PROTOCOL)
m.shape

TypeError: 'protocol' is an invalid keyword argument for this function

In [41]:
with open('cos_sim.npy', 'wb') as f:
    np.save(f, cos_sim)

In [42]:
with open('cos_sim.npy', 'rb') as f:
    a = np.load(f)
a.shape

(15000, 15000)

In [79]:
from sklearn.externals import joblib

C:\Users\Lenovo IdeaPad JQ\AnacondaFolder\envs\justin-env\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [80]:
joblib.dump(cos_sim, 'cos_sim.pkl')

['cos_sim.pkl']

In [81]:
loaded_sim = joblib.load('cos_sim.pkl')
loaded_sim.shape

(25000, 25000)

In [105]:
joblib.dump(cos_sim, 'compressed_cos_sim.pkl', compress=1)

['compressed_cos_sim.pkl']

In [106]:
loaded_sim = joblib.load('compressed_cos_sim.pkl')
loaded_sim.shape

(20000, 20000)

In [23]:
with gzip.open(pickle_filename, 'rb') as f:
    loaded_obj = cPickle.load(f)
loaded_obj.shape

(23000, 23000)

In [24]:
def get_recommendations_v3(title, loaded_obj):
    index = indices[title]
    
    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(loaded_obj[index]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:26]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return metadata_df['title'].iloc[movie_indices]

reco = get_recommendations_v3('Mean Girls', loaded_obj)
reco

13775              The Clique
1296                Cape Fear
15086      Death at a Funeral
1295                Cape Fear
3675             The In Crowd
20402             Now Is Good
19260                  Bag It
21832      Dances with Dragon
10722          A Woman's Face
11527                    Time
4087     Revenge of the Nerds
10596             The Kid & I
5617     Happy Birthday to Me
3563           Prizzi's Honor
6078        Nowhere in Africa
22306              Demon Wind
7097       The Out of Towners
7123           Hell's Kitchen
6571                    Kuffs
22141                    Amen
708                 The Craft
6306          North to Alaska
555                    Killer
7107              Latter Days
17434        Read It and Weep
Name: title, dtype: object